<a href="https://colab.research.google.com/github/cierrakoen/ML_Hotel_Cancellation_Predictor/blob/main/ML_Hotel_cancellations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating Imports

In [ ]:
import seaborn as sb
import sklearn
import pandas as pd
import numpy as np
import matplotlib
import json
import csv
import os

### Report must include different ML models with 3 different sampling evaluations (50-50, 70-30, and 80-20)


1. Decision Tree/Random Forest - Ike
2. Perceptron - Cierra 
3. Naive Bayes - Omari
4. Logistic Regression - Ike
5. Linear Regression - Cierra 
6. SVM - Linear Kernel - Omari 
7. SVM - RBF kernel - Ike
8. Gradient Boosting - Cierra 
9. Multi-layer Perceptron (MLP) - Omari



Information on how to get the data set: https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/



*   FYI: all downloads will be found under content 
*   Don't forget to remove your API key (kaggle.json)



In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download jessemostipak/hotel-booking-demand
! kaggle datasets download jessemostipak/hotel-booking-demand

In [ ]:
! unzip hotel-booking-demand.zip

Archive:  hotel-booking-demand.zip
  inflating: hotel_bookings.csv      


#Creating a way to read the csv

# Creating X and y values
 